In [1]:
from tqdm import tqdm
from typing import List, Dict
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation,PacketFlowRepressentation
from flowprintOptimal.sekigo.dataAnalysis.vNATDataFrameProcessor import VNATDataFrameProcessor
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
import random
from flowprintOptimal.sekigo.flowUtils.flowDatasets import PacketFlowDataset
from torch.utils.data import Dataset,DataLoader
from torchsampler import ImbalancedDatasetSampler
from sklearn.model_selection import train_test_split
from flowprintOptimal.sekigo.flowUtils.commons import normalizePacketRep
import os
from joblib import Parallel, delayed
from flowprintOptimal.sekigo.flowUtils.commons import saveFlows,loadFlows
from flowprintOptimal.sekigo.dataAnalysis.dataFrameProcessor import UTMobileNetProcessor
from flowprintOptimal.sekigo.flowUtils.dataGetter import getTrainTestOOD
from sklearn.metrics import confusion_matrix
import json
from flowprintOptimal.sekigo.modeling.trainers import NNClassificationTrainer
from flowprintOptimal.sekigo.modeling.neuralNetworks import LSTMNetwork,TransformerGenerator,CNNNetwork1D,LSTMDuelingNetwork
from flowprintOptimal.sekigo.modeling.loggers import Logger
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
from flowprintOptimal.sekigo.earlyClassification.DQL.core import MemoryElement,Rewarder,State
from flowprintOptimal.sekigo.earlyClassification.DQL.memoryFiller import MemoryFiller
from flowprintOptimal.sekigo.earlyClassification.DQL.datasets import MemoryDataset
from flowprintOptimal.sekigo.earlyClassification.DQL.trainers import EarlyClassificationtrainer
from flowprintOptimal.sekigo.utils.documentor import Documenter
from flowprintOptimal.sekigo.utils.evaluations import Evaluator,EarlyEvaluation
from flowprintOptimal.sekigo.flowUtils.commons import getTimeStampsFromIAT, getIATFromTimeStamps
import heapq
import warnings
warnings.filterwarnings('ignore')
import torch.nn as nn
from torch.distributions.categorical import Categorical
from joblib import Parallel, delayed
from flowprintOptimal.sekigo.dataAnalysis.dataFrameProcessor import MirageProcessor


In [74]:
processor = MirageProcessor(data_path= "data/MIRAGE-2019")

In [86]:
packet_reps = processor.getPacketReps(mode= "app_catagory")

In [89]:
lengths = list(map(lambda x : len(x), packet_reps))

pd.Series(lengths).describe()

count    117884.000000
mean         23.023727
std           9.785366
min           1.000000
25%          17.000000
50%          26.000000
75%          32.000000
max          32.000000
dtype: float64

In [87]:
labels = list(map(lambda x :x.class_type, packet_reps))

labels = pd.Series(labels)

In [88]:
saveFlows(path= "data/MIRAGE-2019/miragePacketRepAppCatagory.json", flows= packet_reps)

In [67]:
d["flow_metadata"]

{'BF_label': 'air.com.hypah.io.slither',
 'BF_labeling_type': 'most-common',
 'BF_num_packets': 53,
 'BF_IP_packet_bytes': 22979,
 'BF_L4_payload_bytes': 20183,
 'BF_duration': 324.4061441421509,
 'UF_num_packets': 25,
 'UF_IP_packet_bytes': 14416,
 'UF_L4_payload_bytes': 13108,
 'UF_duration': 324.4061441421509,
 'DF_num_packets': 28,
 'DF_IP_packet_bytes': 8563,
 'DF_L4_payload_bytes': 7075,
 'DF_duration': 324.3795208930969}

In [54]:
timesteps = getTimeStampsFromIAT(inter_arrival_times= packet_reps[0].inter_arrival_times)

In [57]:
(timesteps[-1] - timesteps[0]).total_seconds()

1.235999

In [52]:
packet_reps[0].inter_arrival_times

[0.0,
 0.7569784956184226,
 0.18797258507353948,
 0.7139942765287248,
 0.8721721424115088,
 0.7544237176041286,
 0.18104646062300564,
 0.7517924577561939,
 0.7186748617051665,
 0.7414242726397329,
 0.7014076033248066,
 0.554326461930472,
 0.7324846050830552,
 0.7147267086840452,
 0.9656542376894935,
 0.5544308315720162,
 0.6049393970169734,
 0.5841201434032341,
 0.6613634740226954,
 0.6345059499165621,
 0.7185677990432882,
 0.15918339406838902,
 0.6585242868193684,
 0.7974487310669145,
 0.6212037130292398,
 0.6994679742887879,
 0.8800992601779078,
 0.18104646062300564,
 0.174989265521228,
 0.174989265521228,
 0.5691390417892545,
 0.5038579082034343]